## Random Acts of Pizza Kaggle Demo

###James Route, Eric Whyne, Raymond Buhr, Filip Krunic
This project is based on the [Random Acts of Pizza](https://www.kaggle.com/c/random-acts-of-pizza) Kaggle competition. The competition is derived from Reddit's Random Acts of Pizza board, where users post a request and short message in hopes that someone else will read it and order them a pizza. The competition's objective is to accurately predict whether a post will successfully receive a pizza using a limited amount of data: the text of the post and title, and several metadata fields such as the post's timestamp and the number of posts the user has made.

We are given approximately 5500 sample posts in JSON format to work with. Of these, 4000 are reserved for training and the remaining 1500 are for testing. The test set has fewer data fields and does not include a field with the correct answers, so for much of this notebook we reserve about 500 samples in a dev set for checking accuracy.

To start with, we import all of the libraries we'll need. This notebook assumes that the files containing training and test data (train.json and test.json) are located in the same directory.

In [1]:
# General libraries for reading and manipulating data
import re
import numpy as np
import json
import datetime
from collections import Counter

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.mixture import GMM
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.neural_network import BernoulliRBM

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# import nltk functions for manipulating text
import nltk
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

### Load data from file and function for Kaggle format

In this block of code we set up data structures, divide the data into train and dev sets, and create a function to write files to the kaggle format.

In [86]:
orig_train = json.loads(open("train.json").read())
orig_test = json.loads(open("test.json").read())

all_labels = []
for item in orig_train:
    all_labels.append(int(item['requester_received_pizza']))
all_labels = np.array(all_labels)


# Create a function for writing solutions to a file in kaggle format
def write_solution_to_file(prediction, filename):
    if len(prediction) == len(kaggle_test_id):
        print "Writing to file", filename
        outfile = open(filename,'w')
        outfile.write("request_id,requester_received_pizza\n")
        for i in range(0,len(prediction)):
            outfile.write(kaggle_test_id[i]+','+str(prediction[i])+'\n')
        return 1
    else:
        print "Prediction dimension mismatch"
        return 0

### A quick experiment

In [87]:
print 100 - sum(all_labels) / float(len(all_labels)) * 100, "% of requestors received pizza in our training set."
print "We could probably get an interesting score just by submitting zeros. Let's do that first."
write_solution_to_file(np.zeros(len(orig_test)),"submission_zeros.csv")
print "Submitting all zeros to Kaggle resulted in a 0.50000 score."

75.396039604 % of requestors received pizza in our training set.
We could probably get an interesting score just by submitting zeros. Let's do that first.
Writing to file submission_zeros.csv
Submitting all zeros to Kaggle resulted in a 0.50000 score.


The fact that submitting all zeros to Kaggle returned a score of .5 tells us that their test data set is comprised of examples evenly split between receiving and not receiving pizza. 

###Part 1: Initial model using text-based approach
We make our first attempt at training a model for prediction. Since this is a text-heavy task, it seems reasonable to try a bag-of-words model using a countVectorizer for feature transformation and logistic regression for prediction.

In [58]:
# these lists store the text from each request and the outcome of the request
text = []

for item in orig_train:
    # concatenate the text and title, then take all unique tokens
    text.append(item['request_text_edit_aware'] + ' ' + item['request_title'])
    
f.close()

# separate data into training and dev sets. this gives us ~500 samples in dev
train_labels = np.array(all_labels[:3500])
train_text = np.array(text[:3500])

dev_labels = np.array(all_labels[3500:])
dev_text = np.array(text[3500:])

In [40]:
# set up a basic vectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_text)
Y = vectorizer.transform(dev_text)    
print X.shape, Y.shape
    
# fit a logistic regression model and test accuracy on dev
lm = LogisticRegression()
lm.fit(X, train_labels)
pred_labels = lm.predict(Y)
    
score = metrics.f1_score(dev_labels, pred_labels)
print "base f1 =", score

correct = (pred_labels == dev_labels)
accuracy = 1.0 * np.sum(correct) / len(pred_labels)
print "%.03f" %accuracy

(3500, 12319) (540, 12319)
base f1 = 0.25
0.700


## Part 2: Switching text method for numerical data
We can try to reduce the vocabulary size to make this model more generalizable, using stemming/lemmatization, stopword elimination, filtering with regular expressions, etc. We can also try other models, such as Naive Bayes. However, none of these offer big improvements.

Therefore, a purely text-based approach must not be the answer. This time we try using the numerical metadata in each of the samples. Note that we only use the fields that are available in both the training and test files.

In [65]:
def meta_to_train(data):
    # store the metadata fields in each sample
    metadata = []
    for item in data:   
        # extract metadata. store data for each post in temporary list
        sample_data = []
        sample_data.append(item['requester_account_age_in_days_at_request'])
        sample_data.append(item['requester_days_since_first_post_on_raop_at_request'])
        sample_data.append(item['requester_number_of_comments_at_request'])
        sample_data.append(item['requester_number_of_comments_in_raop_at_request'])
        sample_data.append(item['requester_number_of_posts_at_request'])
        sample_data.append(item['requester_number_of_subreddits_at_request'])
        sample_data.append(item['requester_number_of_posts_on_raop_at_request'])
        sample_data.append(item['requester_upvotes_minus_downvotes_at_request'])
        sample_data.append(item['requester_upvotes_plus_downvotes_at_request'])   
        metadata.append(sample_data)
    return np.array(metadata)

orig_train_metadata = meta_to_train(orig_train)
kaggle_test_metadata = meta_to_train(orig_test)

train_metadata = orig_train_metadata[:3500]
dev_metadata = orig_train_metadata[3500:]
print "Our training and dev sets shape:", train_data.shape, dev_data.shape
print "Kaggle test shape", kaggle_test_metadata.shape

Our training and dev sets shape: (3500, 9) (540, 9)
Kaggle test shape (1631, 9)


Now we try a few simple models and see how the performance looks.

In [66]:
# train k-nearest numbers model using the metadata
def trainKNN(k=1):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(train_metadata, train_labels)
    pred_labels = model.predict(dev_metadata)
    
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(pred_labels)
    print "K-nearest neighbors:"
    print "%.03f" %accuracy
    
    score = metrics.f1_score(dev_labels, pred_labels)
    print "base f1 =", score

# train logistic regression using the metadata
def trainLM():
    lm = LogisticRegression()
    lm.fit(train_metadata, train_labels)
    pred_labels = lm.predict(dev_metadata)
    
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(pred_labels)
    print "Logistic regression:"
    print "%.03f" %accuracy
    
    score = metrics.f1_score(dev_labels, pred_labels)
    print "base f1 =", score

# train a simple BernoulliNB model
def trainNB(a=1.0):
    bern = BernoulliNB(alpha=a)
    bern.fit(train_metadata, train_labels)
    pred_labels = bern.predict(dev_metadata)
    
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(pred_labels)
    print "Naive Bayes:"
    print "%.03f" %accuracy
    
    score = metrics.f1_score(dev_labels, pred_labels)
    print "base f1 =", score

trainKNN(k=20)
trainLM()
trainNB()

K-nearest neighbors:
0.756
base f1 = 0.0149253731343
Logistic regression:
0.757
base f1 = 0.0839160839161
Naive Bayes:
0.717
base f1 = 0.274881516588


In [69]:
# Logistic regression looked ok, so let's make a Kaggle submission based on that
lm = LogisticRegression()
lm.fit(orig_train_metadata, all_labels) # we want to train with all the data we have when preparing a submission
kaggle_metadata_pred_labels = lm.predict(kaggle_test_metadata)
write_solution_to_file(kaggle_metadata_pred_labels, "submission_metadata_logistic_regression.csv")

# This scored 0.51161 according to kaggle.

Writing to file metadata_logistic_regression_submission.csv


1

## Part 3: Engineering new features for a combined approach
The accuracy of these models is higher, but only NB offers an improvement in f-score. Let's try to get more creative with our feature set and combine some of the text-based features with the metadata. See below for an explanation of the new fields:

* Extraction of keywords as features. A paper accompanying the Kaggle competition from Althoff, et al performed some topic modeling and found sets of keywords that improved prediction accuracy. We implemented these and found slight improvements. What worked better was using Bayes' theorem to find general keywords predictive of pizza. See the appendix for details on what we did.
* Also from Althoff et al, a request in the first half of a month tends to do better. We code this as a binary feature.
* We code the length of the request and the title (in tokens) as features.
* We check the original text file to see if 'jpg' is present, an indicator that the user linked to an image. Note that other image types didn't yield predictive power.
* We code the presence of a hyperlink as another feature
* We convert some of the numerical features to binary or divide by 100 to reduce their size, which improves accuracy. We originally tried normalizing all data so it would fall between 0 and 1, which reduced accuracy. We also tried a log transform to correct high skew in some of the fields, which also didn't work.

In [7]:
# Use the nltk downloader and select stopwords wordnet from the corpora section.  
nltk.download()

# Be sure to close the downloader window before executing the rest of the notebook.

showing info http://www.nltk.org/nltk_data/


True

In [71]:
def train_to_combined(data):
    # keywords to extract as features
    highprob = [u'surviving', u'stretch', u'married', u'total', u'incredibly', u'landlord', 
            u'cover', u'receiving', u'stopped', u'nj', u'bare', u'current', u'jpg', 
            u'exchange', u'normally', u'applied', u'checks', u'truly', u'heat', u'aren', 
            u'plain', u'kentucky', u'reasons', u'father', u'rice', u'including', u'talking', 
            u'especially', u'second', u'costs', u'july', u'lift', u'posted', u'considering', 
            u'eggs', u'generosity', u'meals', u'certainly', u'bucks', u'mac', u'fall', 
            u'rather', u'receive', u'deal', u'aid', u'tight', u'imgur', u'lots', u'tough', 
            u'eternally', u'f', u'expenses', u'beans', u'savings', u'visit', u'daughter', 
            u'accident', u'call', u'form', u'boys', u'available', u'seem', u'checking', 
            u'details', u'cheap', u'financially', u'assistance', u'non', u'unexpected', 
            u'sunday', u'oatmeal', u'pic']

    # list to store extracted features
    metadata = []

    # tokenize using this regex
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')

    # lemmatize the tokens
    wl = WordNetLemmatizer()

    # filter hyperlinks with this regex
    links = re.compile(r'\bhttp\S+\b')

    # stopwords to filter with nltk
    sw = stopwords.words('english')

    for item in data:
        
        # store data for each sample in temporary list
        num = []
        
        # get the text of request
        words = unicode(item['request_text_edit_aware'])
        
        # remove links and replace with string "hyperlink." this prevents lots of spurious tokens
        words = links.sub(u'hyperlink', words)
        
        # remove stopwords, tokenize, lemmatize
        tokens = [t for t in tokenizer.tokenize(words) if t.lower() not in sw]
        tokens = [wl.lemmatize(t.lower()) for t in tokens]

        # convert tokens back to string and store it
        #text.append(' '.join(tokens))
        
        # get the title of request, tokenize, stem/lemmatize
        title_words = unicode(item['request_title'])
        title_tokens = [t for t in tokenizer.tokenize(title_words) if t.lower() not in sw]
        title_tokens = [wl.lemmatize(t.lower()) for t in title_tokens]
        
        # store the number of tokens in the request and title as additional features
        num.append(len(tokens))
        num.append(len(title_tokens))
    
        # combine tokens from text and title
        tokens += title_tokens
        
        # extract metadata. store data for each request in list
        num.append(item['requester_account_age_in_days_at_request']/100.0)
        num.append(item['requester_days_since_first_post_on_raop_at_request'])
        num.append(item['requester_number_of_comments_at_request']/100.0)
        num.append(int(item['requester_number_of_comments_in_raop_at_request'] > 0))
        num.append(item['requester_number_of_posts_at_request'])
        num.append(item['requester_number_of_subreddits_at_request'])
        num.append(int(item['requester_number_of_posts_on_raop_at_request'] > 0))
        num.append(item['requester_upvotes_minus_downvotes_at_request'])
        num.append(item['requester_upvotes_plus_downvotes_at_request'])
        
        # extract one feature for each word in the keyword array
        for word in highprob:
            num.append(int(word in tokens))
        
        # manually check for jpg in the request, since we already filtered urls
        if 'jpg' in item['request_text_edit_aware']:
            num.append(1)
        else:
            num.append(0)
        
        # if the text contains a hyperlink, set feature to 1
        num.append(int('hyperlink' in tokens))
       
        # get the day of the month request occurred. set feature to 1 if in first half of month
        dt = int(datetime.datetime.fromtimestamp(item['unix_timestamp_of_request_utc']).strftime('%d'))
        num.append(int(dt <= 15))
                
        # append the entire list to num_data, which stores data for all examples
        metadata.append(num)
    return np.array(metadata)

combined_features = train_to_combined(orig_train)
kaggle_test_combined_features = train_to_combined(orig_test)

train_data = combined_features[:3500]
dev_data = combined_features[3500:]
print train_data.shape, dev_data.shape

(3500, 86) (540, 86)


###3.1: Optimizing models for better performance
Next, we try running a set of algorithms again and compare the scores. Note that the acccuracy and f1 for the regression continues to rise, as does the f1 for Naive Bayes. We further increase this by using gridsearch to optimize parameter values for some of the models.

In [96]:
# run gridsearch to find best k value
def findK():
    # range is from an iterative process to narrow down
    params = {'n_neighbors': range(38,56,1)}
    search = GridSearchCV(KNeighborsClassifier(), params)
    search.fit(train_data, train_labels)
    
    print "the best parameter is k=%f\n" %search.best_params_['n_neighbors']
    #print "summary of all params:\n", search.grid_scores_

def bernParams():
    #params = {'binarize': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
    params = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 20.0, 50.0]}
    search = GridSearchCV(BernoulliNB(), params)
    search.fit(train_data, train_labels)
    
    print "The best parameter is alpha=%f\n" %search.best_params_['alpha']
    #print "summary of all params:\n", search.grid_scores_

print "KNN"
findK()
trainKNN(k=49)

print "\nNB"
bernParams()
trainNB(a=20.0)

KNN
the best parameter is k=49.000000

K-nearest neighbors:
0.757
base f1 = 0.0

NB
The best parameter is alpha=20.000000

Naive Bayes:
0.717
base f1 = 0.274881516588


In [99]:
# Make a sumbission using KNN where K=49
model = KNeighborsClassifier(n_neighbors=49)
model.fit(orig_train_metadata, all_labels)
kaggle_metadata_pred_labels = model.predict(kaggle_test_metadata)

write_solution_to_file(kaggle_metadata_pred_labels, "submission_knn49.csv")

# This solution produced a result of 0.50124 on kaggle.

Writing to file submission_knn49.csv


1

In [89]:
def trainAB(): 
    model = AdaBoostClassifier()
    model.fit(train_data, train_labels)
    pred_labels = model.predict(dev_data)
    
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for AdaBoostClassifier is %.03f" %accuracy
    
# PCA randomForest pipeline
def pcaTrainRF():
    model = RandomForestClassifier()
    pca = PCA(n_components = 5, whiten=True)
    pipe = Pipeline(steps=[('pca', pca), ('RF', model)])
    pipe.fit(train_data, train_labels)
    pred_labels = pipe.predict(dev_data)
    
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for PCA + RandomForest is %.03f" %accuracy
    
# BernoulliRBM + logistic regression
def rbmLogistic():
    model = SVC()
    rbm = BernoulliRBM(random_state=0, verbose=False)
    rbm.learning_rate=0.86
    rbm.n_iter = 20
    rbm.n_components = 100
    pipe = Pipeline(steps=[('rbm', rbm), ('model', model)])
    pipe.fit(train_data, train_labels)
    pred_labels = pipe.predict(dev_data)
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for RBM + SVM is %.03f" %accuracy    

# random forest classifier 
def trainRF():
    model = RandomForestClassifier()
    model.fit(train_data, train_labels)
    pred_labels = model.predict(dev_data)
    
    # display the accuracy
    correct = (pred_labels == dev_labels)
    accuracy = 1.0 * np.sum(correct) / len(dev_labels)
    print "accuracy for RandomForest is %.03f" %accuracy
    
trainRF()
rbmLogistic()
trainAB()
pcaTrainRF()

accuracy for RandomForest is 0.750
accuracy for RBM + SVM is 0.757
accuracy for AdaBoostClassifier is 0.757
accuracy for PCA + RandomForest is 0.700


##Appendix A: Generating keywords from text data
We initially worked with the idea from Althouse, et al and checked for the presence of words from among several lists. This only succeeded increasing accuracy a marginal amount, so we decided to derive an approach using Bayes' Theorem, determining which words in the entire corpus are more strongly associated with receiving pizza. Below is the code use to accomplish this.

In [ ]:
p_count = Counter() # holds number of successful requests that contain a given word
w_count = Counter() # holds number of total requests that contain a given word
probs = [] # holds the probabilities associated with each word
total = 0

f = open("train.json", "r")
#tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
#sw = stopwords.words('english')
for item in data:
        
    # extract the outcome labels and text for request and title
    receive = int(item['requester_received_pizza'])
    words = item['request_text_edit_aware'] + ' ' + item['request_title']
    tokens = set([t.lower() for t in tokenizer.tokenize(words) if t.lower() not in sw])
        
    if receive == 1:
        p_count.update(t for t in tokens)
        
    w_count.update(t for t in tokens)
    total += 1

f.close()

# store results in tuple. contents are the word, occurrences in successful requests, ratio of
# occurrences in successful request to total occurrences
for key, value in p_count.iteritems():
    prob = 1.0 * value / w_count.get(key)
    probs.append((key, value, prob))

probs.sort(key=lambda tup: tup[2], reverse=True)

Now we can look at the sorted list and pick out the terms that are most strongly associated with receiving pizza, based on number of occurrances and probability. Setting lower thresholds tends to yield the best results on the dev data, but when submitting to Kaggle, more restrictive lists work better.

In [64]:
# print contents of the tuple list based on conditions
wordlist = []

for item in probs:
    # print tuple if word occurs in at least 10 successful requests and 45% of occurrences are in successful requests
    if item[1] >= 15 and item[2] >= .4:
        #print item
        wordlist.append(item[0])

# this just prints the words. handy if you want to paste list directly into code
print wordlist

[u'cover', u'current', u'jpg', u'exchange', u'normally', u'aren', u'father', u'rice', u'especially', u'second', u'posted', u'considering', u'meals', u'bucks', u'rather', u'receive', u'deal', u'aid', u'tight', u'imgur', u'tough', u'expenses', u'beans', u'visit', u'daughter', u'call', u'checking', u'details', u'cheap', u'financially', u'assistance', u'sunday']


##Appendix B: The cutting room floor
There were a number of features and other experiments we tried that didn't work out. These are listed here to give a sense of the other things we tried that didn't positively impact prediction accuracy:

* Normalizing numeric data so it is contained on the interval [0,1]
* Using log and inverse transforms to correct skew in numeric features
* Extracting other features from the timestamp, such as time, day of the week, month, etc.
* Using textBlob to calculate sentiment analysis and code it as a feature
* Using NLTK to perform part-of-speech tagging and coding occurrences of certain words as features (proper nouns, adjectives, adverbs, etc.)